<a href="https://colab.research.google.com/github/mozzaquatro/financial_market/blob/main/08_Analise_Fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

2. Obtendo e tratando os dados

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [4]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [5]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
1,CSTB3,150.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
2,PORP4,2.40,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.0,2.239900e+07,0.00,"13,66%"
3,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
4,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
980,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
981,GLOB4,30.49,4873.60,8.59,1.512,"0,00%",1.389,-63.21,46.91,-4.59,55.03,28.77,"3,22%","0,03%",0.96,"3,90%","0,18%",0.0,5.675000e+09,1.76,"5,35%"
982,LWSA3,12.92,18050.70,2.58,10.968,"0,21%",1.813,4.51,241.77,7.37,187.94,54.36,"4,54%","0,06%",3.82,"1,33%","0,01%",163323000.0,2.950560e+09,0.04,"53,23%"


tratando os dados

In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float')/100

In [7]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CLAN3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0105,0.0,1.012240e+09,0.00,-0.6396
1,CSTB3,150.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.0,8.420670e+09,0.14,0.3191
2,PORP4,2.40,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0208,0.0,2.239900e+07,0.00,0.1366
3,PMET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.0,-2.908630e+08,0.00,0.3774
4,MNSA3,0.42,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.0,-9.105000e+06,-6.52,-0.4111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,UBBR11,14.75,1201.81,3.91,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
980,UBBR3,18.00,1466.61,4.77,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
981,GLOB4,30.49,4873.60,8.59,1.512,0.0000,1.389,-63.21,46.91,-4.59,55.03,28.77,0.0322,0.0003,0.96,0.0390,0.0018,0.0,5.675000e+09,1.76,0.0535
982,LWSA3,12.92,18050.70,2.58,10.968,0.0021,1.813,4.51,241.77,7.37,187.94,54.36,0.0454,0.0006,3.82,0.0133,0.0001,163323000.0,2.950560e+09,0.04,0.5323


**Analisando os dados**

empresa com liquidez 
1.000.000 média diária dos ultimos 2 meses

In [8]:
df_liq2 = df[df['Liq.2meses'] > 1000000]

In [9]:
df_liq2.shape

(281, 21)

In [10]:
# 1. P/L (preço da ação dividido pelo lucro por ação).
# não pode ser muito alto (ex.: 10,20,...)
# entra 0 e 10
df_pl = df[df['P/L'].between(0,10)]

In [11]:
df_pl

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CLAN3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0105,0.00,1.012240e+09,0.00,-0.6396
1,CSTB3,150.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.00,8.420670e+09,0.14,0.3191
2,PORP4,2.40,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0208,0.00,2.239900e+07,0.00,0.1366
3,PMET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.00,-2.908630e+08,0.00,0.3774
4,MNSA3,0.42,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.00,-9.105000e+06,-6.52,-0.4111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,VIVO4,71.80,9.81,2.93,1.491,0.0000,1.282,-31.87,6.78,-5.17,7.22,7.22,0.2201,0.1520,0.89,0.2392,0.2988,0.00,9.816290e+09,0.35,0.1306
663,FESA3,50.00,9.95,2.02,2.034,0.0149,1.412,5.07,7.00,14.32,6.91,5.54,0.2904,0.2044,3.33,0.2462,0.2030,19393.70,2.185840e+09,0.19,0.1468
664,CSAB4,42.98,9.95,1.82,105.090,0.0476,0.882,0.00,11.34,0.00,0.00,0.00,9.2703,10.5600,0.00,0.0000,0.1832,1558.50,1.810920e+08,0.00,-0.1563
665,TCSL4,7.82,9.95,0.83,1.105,0.0000,0.475,8.93,5.69,-2.20,5.61,2.18,0.1944,0.1111,1.34,0.0967,0.0833,0.00,2.284780e+10,0.14,0.0229


menor relacao EV/Ebit e
maior ROI

In [9]:
ranking = pd.DataFrame()
ranking['pos']=range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [10]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,CMIN3
1,2,GOAU3,WIZS3
2,3,GOAU4,PSSA3
3,4,USIM3,TASA4
4,5,EVEN3,TASA3
...,...,...,...
145,146,VULC3,SEER3
146,147,VIIA3,DIRR3
147,148,ALSO3,EZTC3
148,149,LREN3,COCE5


In [ ]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [ ]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [ ]:
t = pd.concat([a,b])
t

,AALR3,ABEV3,ALLD3,ALSO3,ALUP11,AMAR3,ASAI3,AURA33,B3SA3,BEEF3,BLAU3,BMOB3,BRFS3,BRKM5,BRML3,CAML3,CARD3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,COCE5,CPFE3,CPLE11,CPLE3,CPLE6,CRFB3,CRPG5,CSED3,CSMG3,CSNA3,CURY3,CYRE3,DEXP3,DIRR3,DMMO3,DXCO3,ECOR3,...,TASA3,TASA4,TECN3,TEND3,TGMA3,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP3,UNIP6,USIM3,USIM5,VALE3,VBBR3,VIIA3,VITT3,VIVT3,VLID3,VULC3,WIZS3,AERI3,ALPA4,AMBP3,ARZZ3,BOAS3,INTB3,MOSI3,MULT3,ORVR3,PETZ3,RADL3,RDOR3,RRRP3,TFCO4,VAMO3,VIVA3,WEGE3
pos,144.0,139,26,147.0,21,96,116,76.0,102,51,121,87,115,10,137.0,86,81,112,80,15,75,59,11,106,66,48,45,49,88,94,97,54,9,47,98,61,55,22,91,73,...,28,27,33,67,84,82,141,72,38,74,133,37,36,5,6,8,100,146.0,126,136.0,107,140,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,NaN,62,46,NaN,40,118,57,NaN,37,36,14,48,127,15,NaN,84,47,110,133,126,108,107,1,143,86,124,123,125,98,28,136,101,11,34,137,55,141,12,69,79,...,5,4,83,121,59,120,49,117,63,91,139,9,8,17,16,6,80,NaN,43,NaN,148,129,2,114.0,39.0,144.0,96.0,95.0,56.0,38.0,149.0,150.0,105.0,99.0,111.0,112.0,25.0,82.0,53.0,30.0


In [ ]:
rank = t.dropna(axis=1).sum()
rank

ABEV3     201
ALLD3      72
ALUP11     61
AMAR3     214
ASAI3     173
         ... 
VBBR3     180
VITT3     169
VLID3     255
VULC3     269
WIZS3      18
Length: 133, dtype: int64

In [ ]:
rank.sort_values()[:15]

PSSA3     4
CMIN3    12
VALE3    14
WIZS3    18
CSNA3    20
ETER3    21
USIM5    22
USIM3    22
BRKM5    25
GOAU3    26
GOAU4    26
MRFG3    29
TASA4    31
TASA3    33
GGBR3    34
dtype: int64